   # Rossmann Store Sales — Modelo (XGBoost, LightGBM & MLP Ensemble)







   Feature Engineering, XGBoost, LightGBM y MLP (con embeddings). Agregamos otpimización de los hiperparámetros de XGBoost usando Optuna. 





In [ ]:
import os
import numpy as np
import pandas as pd
from xgboost import XGBRegressor, DMatrix
import optuna
import json
import joblib 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# --- CONFIGURACIÓN IMPORTANTE ---
RUN_OPTUNA = True # Optimizar con Optuna
LOAD_HYPERPARAMS = False  # Si hay archivo de hiperparámetros, cargarlo
RETRAIN_WITH_FULL_TRAIN_SET = True  # Reentrenar el modelo final usando todo el conjunto de entrenamiento
RUN_LGBM = False # Ejecutar LightGBM
RUN_MLP = False # Ejecutar MLP
USE_EMBEDDINGS = False # Usar embeddings en MLP

HYPERPARAM_FILE = 'best_params.json'
XGB_MODEL_FILE = 'xgb_model_final.joblib'
LGBM_MODEL_FILE = 'lgbm_model_final.joblib'
MLP_MODEL_FILE = 'mlp_model_final.h5'
SCALER_FILE = 'scaler_final.joblib'



/opt/conda/envs/dev/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-12 20:28:19.764413: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-12 20:28:19.847347: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-12 20:28:21.810299: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different num

   ## 1. Carga de datos

In [2]:
BASE = "dataset/rossmann"
train = pd.read_csv(os.path.join(BASE, "train.csv"), low_memory=False)
test  = pd.read_csv(os.path.join(BASE, "test.csv"),  low_memory=False)
store = pd.read_csv(os.path.join(BASE, "store.csv"), low_memory=False)
train["Date"] = pd.to_datetime(train["Date"])
test["Date"]  = pd.to_datetime(test["Date"])
test_ids = test["Id"]
print(list(train.columns))



['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']


   ## 2. Merge con `store.csv`

In [3]:
train = train.merge(store, on="Store", how="left")
test  = test.merge(store, on="Store", how="left")
test['Open'] = test['Open'].fillna(1).astype(int)
print(list(train.columns))



['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']


   ## 3. Feature engineering

In [ ]:
def add_date_features(df):
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    df["DayOfWeek"] = df["Date"].dt.dayofweek
    df["WeekOfYear"] = df["Date"].dt.isocalendar().week.astype(int) 
    return df

train = add_date_features(train)
test = add_date_features(test)
train['StateHoliday'] = train['StateHoliday'].replace({'0': 0, 'a': 1, 'b': 1, 'c': 1}).astype(int)
test['StateHoliday'] = test['StateHoliday'].replace({'0': 0, 'a': 1, 'b': 1, 'c': 1}).astype(int)

# Manejo de NAs y valores minimos
min_year = train["Year"].min() # Podia ser 1999 como en el curso, pero me dio mejor el score con el minimo real
na_fill = {
    "CompetitionDistance":      100000.0,
    "CompetitionOpenSinceMonth":       1.0,
    "CompetitionOpenSinceYear":  float(min_year),
    "Promo2SinceWeek":          0.0,
    "Promo2SinceYear":          float(min_year),
    "PromoInterval":            "None",
}
for col, val in na_fill.items():
    train[col] = train[col].fillna(val)
    test[col]  = test[col].fillna(val)

# Features de Antigüedad de Competencia y Promoción 
def add_competition_promo_features(df):
    df["CompetitionOpenMonths"] = (12 * (df["Year"] - df["CompetitionOpenSinceYear"]) + (df["Month"] - df["CompetitionOpenSinceMonth"])).clip(lower=0)
    df["Promo2OpenWeeks"] = ((df["Year"] - df["Promo2SinceYear"]) * 52 + (df["WeekOfYear"] - df["Promo2SinceWeek"])).clip(lower=0)
    return df

train = add_competition_promo_features(train)
test  = add_competition_promo_features(test)

# Features de Proximidad a Feriados 
holidays_dt = pd.to_datetime(train[(train['StateHoliday'] != 0) | (train['SchoolHoliday'] == 1)]['Date'].unique())
def get_next_holiday(date):
    future_holidays = holidays_dt[holidays_dt > date]
    return (future_holidays.min() - date).days if len(future_holidays) > 0 else 9999 
def get_days_since_last_holiday(date):
    past_holidays = holidays_dt[holidays_dt < date]
    return (date - past_holidays.max()).days if len(past_holidays) > 0 else 9999 

train['DaysToNextHoliday'] = train['Date'].apply(get_next_holiday)
train['DaysSinceLastHoliday'] = train['Date'].apply(get_days_since_last_holiday)
test['DaysToNextHoliday'] = test['Date'].apply(get_next_holiday)
test['DaysSinceLastHoliday'] = test['Date'].apply(get_days_since_last_holiday)
print("Features de feriados añadidas.")



/tmp/ipykernel_5214/2209831412.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['StateHoliday'] = train['StateHoliday'].replace({'0': 0, 'a': 1, 'b': 1, 'c': 1}).astype(int)
/tmp/ipykernel_5214/2209831412.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test['StateHoliday'] = test['StateHoliday'].replace({'0': 0, 'a': 1, 'b': 1, 'c': 1}).astype(int)


Features de feriados añadidas.


   ## 4. Limpieza final y separación de X e Y (para XGBoost/LightGBM)

In [ ]:
train_xgb = train[(train["Sales"] > 0) & (train["Open"] == 1)].copy()
y_train_xgb = train_xgb["Sales"]
train_xgb = train_xgb.sort_values('Date') 

# Split temporal

cut_off_date = pd.to_datetime('2015-01-01')
train_subset_xgb = train_xgb[train_xgb['Date'] < cut_off_date]
val_subset_xgb = train_xgb[train_xgb['Date'] >= cut_off_date]
y_train_log = np.log1p(train_subset_xgb["Sales"])
y_val_log = np.log1p(val_subset_xgb["Sales"])

ignore_cols = ["Sales", "Customers", "Date", "Id", 
               "CompetitionOpenSinceMonth", "CompetitionOpenSinceYear",
               "Promo2SinceWeek", "Promo2SinceYear"] 

X_train = train_subset_xgb.drop(columns=ignore_cols, errors='ignore')
X_val = val_subset_xgb.drop(columns=ignore_cols, errors='ignore')
X_test = test.drop(columns=ignore_cols, errors='ignore') 

categorical_cols = ["StoreType", "Assortment", "PromoInterval"]
for col in categorical_cols:
    X_train = pd.get_dummies(X_train, columns=[col], prefix=col, drop_first=True)
    X_val = pd.get_dummies(X_val, columns=[col], prefix=col, drop_first=True)
    X_test = pd.get_dummies(X_test, columns=[col], prefix=col, drop_first=True)

X_val = X_val.reindex(columns=X_train.columns, fill_value=0)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# A lgbm no le gustan los nombres de columnas con caracteres especiales
X_train.columns = X_train.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)
X_val.columns = X_val.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)
X_test.columns = X_test.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)

print(f"XGB/LGBM Train shape: {X_train.shape}, X_val shape: {X_val.shape}, X_test shape: {X_test.shape}")



XGB/LGBM Train shape: (648309, 24), X_val shape: (196029, 24), X_test shape: (41088, 24)


   ## 5. Entrenamiento del Modelo XGBoost y Optimización con Optuna

In [ ]:
params_xgb = {
    'objective': 'reg:squarederror', 
    'n_estimators': 5000, 
    'learning_rate': 0.03, 
    'max_depth': 9,
    'seed': 42,
    'n_jobs': -1,
}

if not RETRAIN_WITH_FULL_TRAIN_SET:
  params_xgb['early_stopping_rounds'] = 50

model_xgb = XGBRegressor(**params_xgb)

if RUN_OPTUNA:
    if LOAD_HYPERPARAMS and os.path.exists(HYPERPARAM_FILE):
        with open(HYPERPARAM_FILE, 'r') as f:
            best_params = json.load(f)
        params_xgb.update(best_params)
    elif not LOAD_HYPERPARAMS:
        def objective(trial):
            param = {'max_depth': trial.suggest_int('max_depth', 3, 12), 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
                'n_estimators': 1000, 'subsample': trial.suggest_float('subsample', 0.6, 1.0), 'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 10), 'objective': 'reg:squarederror', 'n_jobs': -1, 'seed': 42}
            xgb_model_trial = XGBRegressor(**param)
            xgb_model_trial.fit(X_train, y_train_log, eval_set=[(X_val, y_val_log)], verbose=50)
            preds = xgb_model_trial.predict(X_val); preds = np.expm1(preds) 
            y_val_actual = np.expm1(y_val_log); mask = y_val_actual > 0
            rmspe = np.sqrt(np.mean(np.square((y_val_actual[mask] - preds[mask]) / y_val_actual[mask])))
            return rmspe
        study = optuna.create_study(direction='minimize'); study.optimize(objective, n_trials=50) 
        with open(HYPERPARAM_FILE, 'w') as f:
            json.dump(study.best_params, f, indent=4)
        params_xgb.update(study.best_params); params_xgb['n_estimators'] = 1000

    model_xgb = XGBRegressor(**params_xgb)
    print(f"Modelo XGBoost configurado con parámetros: {params_xgb}")


if RETRAIN_WITH_FULL_TRAIN_SET:
    print("\nBandera RETRAIN_WITH_FULL_TRAIN_SET es True. Reentrenando XGBoost con todos los datos.")
    X_train_full = pd.concat([X_train, X_val], ignore_index=True)
    y_train_full_log = np.log1p(pd.concat([train_subset_xgb["Sales"], val_subset_xgb["Sales"]], ignore_index=True))
    model_xgb.fit(X_train_full, y_train_full_log, verbose=50)
elif not RUN_OPTUNA:
    print("\nBandera RETRAIN_WITH_FULL_TRAIN_SET y RUN_OPTUNA son False. Entrenando XGBoost con validación.")
    model_xgb.fit(X_train, y_train_log, eval_set=[(X_val, y_val_log)], verbose=50) 



[I 2025-12-12 20:30:33,893] A new study created in memory with name: no-name-14d26413-b0c5-4f81-9e97-f93cae94a007


[0]	validation_0-rmse:0.40412
[50]	validation_0-rmse:0.27380
[100]	validation_0-rmse:0.23529
[150]	validation_0-rmse:0.21029
[200]	validation_0-rmse:0.19386
[250]	validation_0-rmse:0.18407
[300]	validation_0-rmse:0.17816
[350]	validation_0-rmse:0.17382
[400]	validation_0-rmse:0.17120
[450]	validation_0-rmse:0.16950
[500]	validation_0-rmse:0.16800
[550]	validation_0-rmse:0.16711
[600]	validation_0-rmse:0.16626
[650]	validation_0-rmse:0.16583
[700]	validation_0-rmse:0.16539
[750]	validation_0-rmse:0.16516
[800]	validation_0-rmse:0.16486
[850]	validation_0-rmse:0.16465
[900]	validation_0-rmse:0.16448
[950]	validation_0-rmse:0.16436
[999]	validation_0-rmse:0.16425


[I 2025-12-12 20:32:32,537] Trial 0 finished with value: 0.17788499479873685 and parameters: {'max_depth': 11, 'learning_rate': 0.05013612703085543, 'subsample': 0.6786924945805478, 'colsample_bytree': 0.7361829590733328, 'min_child_weight': 3}. Best is trial 0 with value: 0.17788499479873685.


[0]	validation_0-rmse:0.40819
[50]	validation_0-rmse:0.32288
[100]	validation_0-rmse:0.29909
[150]	validation_0-rmse:0.27487
[200]	validation_0-rmse:0.25665
[250]	validation_0-rmse:0.24173
[300]	validation_0-rmse:0.22821
[350]	validation_0-rmse:0.21870
[400]	validation_0-rmse:0.21186
[450]	validation_0-rmse:0.20528
[500]	validation_0-rmse:0.19898
[550]	validation_0-rmse:0.19460
[600]	validation_0-rmse:0.19111
[650]	validation_0-rmse:0.18775
[700]	validation_0-rmse:0.18495
[750]	validation_0-rmse:0.18276
[800]	validation_0-rmse:0.18041
[850]	validation_0-rmse:0.17836
[900]	validation_0-rmse:0.17714
[950]	validation_0-rmse:0.17586
[999]	validation_0-rmse:0.17442


[I 2025-12-12 20:33:55,972] Trial 1 finished with value: 0.19049745189107842 and parameters: {'max_depth': 9, 'learning_rate': 0.027921523735905195, 'subsample': 0.9424127783942385, 'colsample_bytree': 0.9299264886832196, 'min_child_weight': 3}. Best is trial 0 with value: 0.17788499479873685.


...

[0]	validation_0-rmse:0.41063
[50]	validation_0-rmse:0.36694
[100]	validation_0-rmse:0.35346
[150]	validation_0-rmse:0.34551
[200]	validation_0-rmse:0.33981
[250]	validation_0-rmse:0.33563
[300]	validation_0-rmse:0.33115
[350]	validation_0-rmse:0.32710
[400]	validation_0-rmse:0.32415
[450]	validation_0-rmse:0.32102
[500]	validation_0-rmse:0.31831
[550]	validation_0-rmse:0.31520
[600]	validation_0-rmse:0.31167
[650]	validation_0-rmse:0.30872
[700]	validation_0-rmse:0.30589
[750]	validation_0-rmse:0.30323
[800]	validation_0-rmse:0.30033
[850]	validation_0-rmse:0.29791
[900]	validation_0-rmse:0.29455
[950]	validation_0-rmse:0.29236
[999]	validation_0-rmse:0.28991


[I 2025-12-12 22:16:32,090] Trial 48 finished with value: 0.3243837673501276 and parameters: {'max_depth': 4, 'learning_rate': 0.01818414219030669, 'subsample': 0.8374140493525122, 'colsample_bytree': 0.9970905824839981, 'min_child_weight': 8}. Best is trial 42 with value: 0.1747637666315192.


[0]	validation_0-rmse:0.40403
[50]	validation_0-rmse:0.26357
[100]	validation_0-rmse:0.23006
[150]	validation_0-rmse:0.20515
[200]	validation_0-rmse:0.19004
[250]	validation_0-rmse:0.17923
[300]	validation_0-rmse:0.17383
[350]	validation_0-rmse:0.17071
[400]	validation_0-rmse:0.16823
[450]	validation_0-rmse:0.16667
[500]	validation_0-rmse:0.16561
[550]	validation_0-rmse:0.16499
[600]	validation_0-rmse:0.16446
[650]	validation_0-rmse:0.16396
[700]	validation_0-rmse:0.16361
[750]	validation_0-rmse:0.16340
[800]	validation_0-rmse:0.16319
[850]	validation_0-rmse:0.16302
[900]	validation_0-rmse:0.16296
[950]	validation_0-rmse:0.16281
[999]	validation_0-rmse:0.16269


[I 2025-12-12 22:19:10,949] Trial 49 finished with value: 0.17617264147075626 and parameters: {'max_depth': 12, 'learning_rate': 0.04618952825616554, 'subsample': 0.6004052845960328, 'colsample_bytree': 0.6888579843006953, 'min_child_weight': 10}. Best is trial 42 with value: 0.1747637666315192.


Modelo XGBoost configurado con parámetros: {'objective': 'reg:squarederror', 'n_estimators': 1000, 'learning_rate': 0.04634495970564556, 'max_depth': 12, 'seed': 42, 'n_jobs': -1, 'subsample': 0.6307765507024194, 'colsample_bytree': 0.6787271687954318, 'min_child_weight': 6}

Bandera RETRAIN_WITH_FULL_TRAIN_SET es True. Reentrenando XGBoost con todos los datos.


   ## 6. Predicciones y Guardado de Modelo XGBoost

In [11]:
joblib.dump(model_xgb, XGB_MODEL_FILE)
print(f"Modelo XGBoost entrenado guardado en {XGB_MODEL_FILE}.")
preds_test_log_xgb = model_xgb.predict(X_test)
preds_test_xgb = np.expm1(preds_test_log_xgb)



Modelo XGBoost entrenado guardado en xgb_model_final.joblib.


   ## 7. Implementación y Predicciones de LightGBM

In [12]:
if RUN_LGBM:
    print("\nBandera RUN_LGBM es True. Entrenando modelo LightGBM.")
    lgb_train = lgb.Dataset(X_train, y_train_log)
    lgb_eval = lgb.Dataset(X_val, y_val_log, reference=lgb_train)
    params_lgbm = {'objective': 'regression', 'metric': 'rmse', 'boosting_type': 'gbdt', 'n_estimators': 5000, 
                   'learning_rate': 0.03, 'num_leaves': 128, 'seed': 42, 'n_jobs': -1, 'verbose': -1}
    model_lgbm = lgb.train(params_lgbm, lgb_train, valid_sets=lgb_eval,
                           callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=True), lgb.log_evaluation(100)])
    joblib.dump(model_lgbm, LGBM_MODEL_FILE)
    print(f"Modelo LightGBM entrenado y guardado en {LGBM_MODEL_FILE}.")
    preds_test_log_lgbm = model_lgbm.predict(X_test, num_iteration=model_lgbm.best_iteration)
    preds_test_lgbm = np.expm1(preds_test_log_lgbm)
else:
    preds_test_lgbm = np.zeros_like(preds_test_xgb) # Placeholder si no se ejecuta LGBM



   ## 8. Preparación de Datos y Implementación de MLP (Keras/TF)

In [ ]:
if RUN_MLP:
    print("\nDefiniendo y entrenando modelo MLP (Keras).")
    
    train_mlp = train[(train["Sales"] > 0) & (train["Open"] == 1)].copy()
    test_mlp  = test.copy()

    # 1. Label Encoding para Keras Embeddings (necesita enteros secuenciales)
    categorical_cols_mlp = ["Store", "StoreType", "Assortment", "DayOfWeek", "Month", "Year", "PromoInterval", "StateHoliday", "SchoolHoliday"]
    for col in categorical_cols_mlp:
        # Asegurarse de que las categorías sean consistentes entre train y test
        full_data = pd.concat([train_mlp[col], test_mlp[col]], axis=0)
        le = LabelEncoder().fit(full_data.astype(str)) # Convertir a str para evitar problemas de tipos mixtos
        train_mlp[col] = le.transform(train_mlp[col].astype(str))
        test_mlp[col]  = le.transform(test_mlp[col].astype(str))

    # 2. Scaling para columnas numéricas
    numerical_cols_mlp = ["CompetitionDistance", "CompetitionOpenMonths", "Promo2OpenWeeks", 
                        "Day", "WeekOfYear", "DaysToNextHoliday", "DaysSinceLastHoliday"]
    scaler = StandardScaler()

    # Ajustar el scaler solo en los datos de entrenamiento y transformar ambos
    train_mlp[numerical_cols_mlp] = scaler.fit_transform(train_mlp[numerical_cols_mlp])
    test_mlp[numerical_cols_mlp]  = scaler.transform(test_mlp[numerical_cols_mlp])

    # Guardar el scaler
    joblib.dump(scaler, SCALER_FILE)

    # Separación final de datos para MLP
    y_train_mlp = np.log1p(train_mlp["Sales"]).values
    train_mlp = train_mlp.drop(columns=["Sales", "Customers", "Date", "Id"] + ignore_cols, errors='ignore')
    test_mlp = test_mlp.drop(columns=["Sales", "Customers", "Date", "Id"] + ignore_cols, errors='ignore')

    # Asegurarse de que todas las columnas usadas en MLP existen y están en orden
    all_mlp_cols = categorical_cols_mlp + numerical_cols_mlp
    train_mlp = train_mlp[all_mlp_cols]
    test_mlp = test_mlp[all_mlp_cols]

    # Crear inputs como diccionarios o listas de arrays NumPy (para .fit() posterior)
    def create_mlp_inputs(df, cat_cols, num_cols):
        inputs = {}
        for col in cat_cols:
            inputs[col] = df[col].values
        inputs['numerical_input'] = df[num_cols].values
        # Retornamos como lista ordenada por tipo para facilitar el uso en fit si se prefiere
        return [df[col].values for col in cat_cols] + [df[num_cols].values]

    input_list_train = create_mlp_inputs(train_mlp, categorical_cols_mlp, numerical_cols_mlp)
    input_list_test = create_mlp_inputs(test_mlp, categorical_cols_mlp, numerical_cols_mlp)

    print(f"MLP Train shapes: {[i.shape for i in input_list_train]}")
    print(f"MLP Target shape: {y_train_mlp.shape}")

    inputs_list_symbolic = []
    embedded_outputs = []
    
    # Definir entradas y embeddings para columnas categóricas
    for col in categorical_cols_mlp:
        # Calcular el tamaño del vocabulario para el embedding: max valor en datos + 1
        vocab_size = int(pd.concat([train_mlp[col], test_mlp[col]], axis=0).max() + 1)
        # Tamaño del embedding (regla común: raíz cuadrada del vocabulario o un número fijo)
        embed_dim = min(50, int(np.sqrt(vocab_size)) + 1)
        
        input_layer = Input(shape=(1,), name=f'Input_{col}')
        embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim, input_length=1)(input_layer)
        flatten_layer = Flatten()(embedding_layer)
        
        inputs_list_symbolic.append(input_layer)
        embedded_outputs.append(flatten_layer)
        
    # Definir entrada para características numéricas (ya escaladas)
    numerical_input_layer = Input(shape=(len(numerical_cols_mlp),), name='Numerical_Input')
    inputs_list_symbolic.append(numerical_input_layer)

    # Concatenar todas las entradas procesadas (embeddings + numéricas)
    concatenated_inputs = concatenate(embedded_outputs + [numerical_input_layer])

    # Construir el resto de la red densa
    dense1 = Dense(units=128, activation='relu')(concatenated_inputs)
    dropout1 = Dropout(0.2)(dense1)
    dense2 = Dense(units=64, activation='relu')(dropout1)
    output_layer = Dense(units=1, activation='linear')(dense2) # Regresión lineal para predecir log(sales+1)

    # --- CORRECCIÓN DE LA LÍNEA DEL ERROR ---
    # Usamos la lista de Inputs SIMBÓLICOS, NO los arrays de NumPy
    model_mlp = Model(inputs=inputs_list_symbolic, outputs=output_layer) 
    
    model_mlp.compile(optimizer='adam', loss='mean_squared_error') # Optimizamos MSE sobre datos log
    model_mlp.summary()

    print("Modelo MLP compilado. Entrenando...")
    
    # Entrenar el modelo
    model_mlp.fit(
        x=input_list_train, 
        y=y_train_mlp, 
        epochs=50, 
        batch_size=256,
        verbose=1
    )

    model_mlp.save(MLP_MODEL_FILE)
    print(f"Modelo MLP final guardado en {MLP_MODEL_FILE}")

   ## 9. Generación de Archivo de Submission Final (Ensemble)

In [ ]:
# Inicializar arrays de predicción a cero para evitar NameError si un modelo no corre
preds_test_xgb = np.zeros(len(test))
preds_test_lgbm = np.zeros(len(test))
preds_test_mlp = np.zeros(len(test))

# Predicciones XGBoost
if not RETRAIN_WITH_FULL_TRAIN_SET and os.path.exists(XGB_MODEL_FILE):
    model_xgb = joblib.load(XGB_MODEL_FILE)
if 'model_xgb' in locals():
    preds_test_xgb = model_xgb.predict(X_test)
    preds_test_xgb = np.expm1(preds_test_xgb)

# Predicciones LightGBM
if RUN_LGBM and 'model_lgbm' in locals():
    # X_test ya está preparado para XGBoost/LGBM
    preds_test_lgbm = model_lgbm.predict(X_test)
    preds_test_lgbm = np.expm1(preds_test_lgbm)

# Predicciones MLP
if RUN_MLP and os.path.exists(MLP_MODEL_FILE):
    # Cargar el modelo guardado y predecir
    model_mlp = tf.keras.models.load_model(MLP_MODEL_FILE)
    preds_test_mlp_log = model_mlp.predict(input_list_test).flatten()
    preds_test_mlp = np.expm1(preds_test_mlp_log)


# Definir pesos basados en qué modelos se ejecutaron
# Asume pesos iguales si todos están activos. Ajusta esto según el rendimiento real de tus modelos.
num_active_models = sum([1 for run in [RUN_MLP, RUN_LGBM, ('model_xgb' in locals())] if run])
if num_active_models == 0:
    print("Error: Ningún modelo fue ejecutado o cargado para el ensemble.")
    preds_test_ensemble = np.zeros(len(test))
else:
    weight = 1.0 / num_active_models
    weight_xgb = weight if ('model_xgb' in locals()) else 0
    weight_lgbm = weight if RUN_LGBM else 0
    weight_mlp = weight if RUN_MLP else 0

    print(f"Pesos usados en el ensemble: XGB={weight_xgb:.2f}, LGBM={weight_lgbm:.2f}, MLP={weight_mlp:.2f}")

    # Calcular predicción final ponderada
    preds_test_ensemble = (weight_xgb * preds_test_xgb) + (weight_lgbm * preds_test_lgbm) + (weight_mlp * preds_test_mlp)

    # Forzar Sales = 0 donde Open = 0 en el test set
    is_closed = test["Open"] == 0 
    preds_test_ensemble[is_closed] = 0

    # Asegurar que no haya ventas negativas
    preds_test_ensemble = np.clip(preds_test_ensemble, a_min=0, a_max=None)

# Generar archivo de submission para Kaggle
submission = pd.DataFrame({"Id": test_ids, "Sales": preds_test_ensemble})
submission.to_csv("submission_ensemble_final.csv", index=False)
print("Archivo de submissión 'submission_ensemble_final.csv' generado exitosamente.")


Pesos usados en el ensemble: XGB=1.00, LGBM=0.00, MLP=0.00
Archivo de submissión 'submission_ensemble_final.csv' generado exitosamente.
